In [22]:
import pandas as pd
import numpy as np
import re
import subprocess
from pathlib import Path 
import io
import dvc.api, dvc.repo
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [2]:
url = "https://github.com/mostafa-fallaha/hackathon-apps-dvc"
data = dvc.api.read("data/googleplaystore.csv", encoding='utf-8', repo=url)
df = pd.read_csv(io.StringIO(data))
df.sample(10)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
2788,"Savory - Deals,Freebies,Sales",SHOPPING,4.400000,2375,1.1M,100000,Free,0.0,Everyone,Shopping,2016-11-02,3.1,2.3
10406,Puzzles Volvo FH 16 Trucks,FAMILY,4.800000,10,8.2M,1000,Free,0.0,Teen,Puzzle,2016-12-02,1.0,4.0
2149,codeSpark Academy & The Foos,FAMILY,4.100000,4522,57M,500000,Free,0.0,Everyone,Educational;Education,2018-06-21,2.19.01,4.1
9040,DX Simulation for OOO Dx Belt,FAMILY,4.600000,67,31M,10000,Free,0.0,Everyone,Simulation,2018-08-04,1.7,4.1
232,Fast Scanner : Free PDF Scan,BUSINESS,4.500000,103755,14M,10000000,Free,0.0,Everyone,Business,2018-07-11,3.9.2,4.1
5250,Aj.Petra,COMMUNICATION,4.191513,5,14M,100,Free,0.0,Everyone,Communication,2017-09-29,1.0.1,4.1
866,Hotstar,ENTERTAINMENT,4.300000,4885646,Varies with device,100000000,Free,0.0,Teen,Entertainment,2018-08-03,Varies with device,Varies with device
10022,Beard Live Camera Photo Editor,BEAUTY,4.700000,900,30M,5000,Free,0.0,Everyone,Beauty,2018-03-25,1.1,4.0
5504,Guess the Class 🔥 AQW,GAME,4.700000,57,12M,1000,Free,0.0,Everyone,Word,2017-11-01,1.0.0,4.1
7940,Free Resume App,BUSINESS,4.600000,15830,Varies with device,1000000,Free,0.0,Everyone,Business,2018-07-19,Varies with device,Varies with device


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   App               10841 non-null  object 
 1   Category          10841 non-null  object 
 2   Rating            10841 non-null  float64
 3   Reviews           10841 non-null  int64  
 4   Size              10841 non-null  float64
 5   Installs          10841 non-null  int64  
 6   Type              10841 non-null  object 
 7   Price             10841 non-null  float64
 8   Content Rating    10841 non-null  object 
 9   Genres            10841 non-null  object 
 10  Last Updated      10841 non-null  object 
 11  Current Ver       10841 non-null  object 
 12  Android Ver       10841 non-null  object 
 13  Category_encoded  10841 non-null  int32  
dtypes: float64(3), int32(1), int64(2), object(8)
memory usage: 1.1+ MB


### Size

In [7]:
def convert_size(size):
    if 'M' in size:
        return float(size.replace('M', ''))
    elif 'k' in size:
        return float(size.replace('k', '')) / 1024
    return np.nan
df['Size_num'] = df['Size'].apply(lambda x: convert_size(x) if x != 'Varies with device' else np.nan)
# Calculate average size per category
df_sorted = df.sort_values(by='Size_num', ascending=False)
top_5_per_category = df_sorted.groupby('Category').head(3)
category_avg_size = top_5_per_category.groupby('Category')['Size_num'].mean().round(1)
print(category_avg_size)
# Replace 'Varies with device' with the average size of the category
df['Size_num'] = df.apply(
    lambda row: category_avg_size[row['Category']] if pd.isna(row['Size_num']) else row['Size_num'],
    axis=1
)

Category
ART_AND_DESIGN          36.3
AUTO_AND_VEHICLES       83.7
BEAUTY                  50.3
BOOKS_AND_REFERENCE     81.7
BUSINESS                95.7
COMICS                  38.3
COMMUNICATION           64.3
DATING                  74.7
EDUCATION               85.7
ENTERTAINMENT           75.7
EVENTS                  51.0
FAMILY                  99.7
FINANCE                 95.3
FOOD_AND_DRINK          69.3
GAME                   100.0
HEALTH_AND_FITNESS      96.7
HOUSE_AND_HOME          73.7
LIBRARIES_AND_DEMO      83.0
LIFESTYLE               94.0
MAPS_AND_NAVIGATION     66.7
MEDICAL                 97.3
NEWS_AND_MAGAZINES      54.7
PARENTING               92.7
PERSONALIZATION         81.0
PHOTOGRAPHY             84.0
PRODUCTIVITY            69.7
SHOPPING                70.3
SOCIAL                  90.0
SPORTS                  98.3
TOOLS                   82.0
TRAVEL_AND_LOCAL        87.7
VIDEO_PLAYERS           81.7
WEATHER                 46.0
Name: Size_num, dtype: float64


In [8]:
# Convert the numeric size back to string with 'M' for display purposes
df['Size'] = df['Size_num'].apply(lambda x: f"{x:.1f}M")
# Drop the helper column
df.drop(columns=['Size_num'], inplace=True)

In [9]:
def convert_size_to_float(size):
    if 'M' in size:
        return float(size.replace('M', ''))
df['Size'] = df['Size'].apply(convert_size_to_float)

In [10]:
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,0.0,Everyone,Art & Design,2018-01-07,1.0.0,4.0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,2018-01-15,2.0.0,4.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,0.0,Everyone,Art & Design,2018-08-01,1.2.4,4.0
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,0.0,Teen,Art & Design,2018-06-08,Varies with device,4.2
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,0.0,Everyone,Art & Design;Creativity,2018-06-20,1.1,4.4


In [11]:
df.iloc[[866]]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
866,Hotstar,ENTERTAINMENT,4.3,4885646,75.7,100000000,Free,0.0,Teen,Entertainment,2018-08-03,Varies with device,Varies with device


### Android version

In [37]:
df['Android Ver'].value_counts()

Android Ver
4.0                   2879
4.1                   2452
Varies with device    1362
4.4                    992
2.3                    933
5.0                    605
4.2                    394
2.2                    245
3.0                    243
4.3                    243
2.1                    134
1.6                    116
6.0                     60
7.0                     43
2.0                     39
3.2                     36
5.1                     24
1.5                     20
3.1                     10
8.0                      6
7.1                      3
1.0                      2
Name: count, dtype: int64

In [43]:
df['Android Ver'] = df['Android Ver'].replace('Varies with device', '4.0.0')

In [44]:
df.iloc[[866]]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Category_encoded
866,Hotstar,ENTERTAINMENT,4.3,4885646,75.7,100000000,Free,0.0,Teen,Entertainment,2018-08-03,Varies with device,4.0.0,9


In [45]:
df['Android Ver'].value_counts()

Android Ver
4.0      2879
4.1      2452
4.0.0    1362
4.4       992
2.3       933
5.0       605
4.2       394
2.2       245
3.0       243
4.3       243
2.1       134
1.6       116
6.0        60
7.0        43
2.0        39
3.2        36
5.1        24
1.5        20
3.1        10
8.0         6
7.1         3
1.0         2
Name: count, dtype: int64

### label encoding

In [13]:
le = LabelEncoder()
df['Category_encoded'] = le.fit_transform(df['Category'])

In [17]:
df.sample(5)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Category_encoded
2135,Disney Magic Kingdoms: Build Your Own Magical ...,FAMILY,4.3,472584,91.0,10000000,Free,0.0,Everyone,Simulation;Action & Adventure,2018-08-02,3.2.0h,4.0,11
5395,Google Photos,PHOTOGRAPHY,4.5,10847682,84.0,1000000000,Free,0.0,Everyone,Photography,2018-08-01,Varies with device,Varies with device,24
3129,"Expedia Hotels, Flights & Car Rental Travel Deals",TRAVEL_AND_LOCAL,4.1,136633,14.0,10000000,Free,0.0,Everyone,Travel & Local,2018-08-06,18.30.0,5.0,30
10639,Florida Today,NEWS_AND_MAGAZINES,3.3,202,38.0,10000,Free,0.0,Everyone 10+,News & Magazines,2018-06-20,5.9.5,5.0,21
9542,EL AL Flights,TRAVEL_AND_LOCAL,3.1,1563,36.0,100000,Free,0.0,Everyone,Travel & Local,2018-05-24,4.3,4.2,30


In [18]:
df['Category_encoded'].max()

32

In [21]:
len(df['Category'].unique())

33

### get dummies

In [46]:
df['Content Rating'].value_counts()

Content Rating
Everyone           8715
Teen               1208
Mature 17+          499
Everyone 10+        414
Adults only 18+       3
Unrated               2
Name: count, dtype: int64

In [47]:
df['Type'].value_counts()

Type
Free    10041
Paid      800
Name: count, dtype: int64

In [52]:
df = pd.get_dummies(df, columns=['Type', 'Content Rating']) #adults only +18 dropped

In [53]:
df.head()

,App,Category,Rating,Reviews,Size,Installs,Price,Genres,Last Updated,Current Ver,Android Ver,Category_encoded,Type_Free,Type_Paid,Content Rating_Adults only 18+,Content Rating_Everyone,Content Rating_Everyone 10+,Content Rating_Mature 17+,Content Rating_Teen,Content Rating_Unrated
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,0.0,Art & Design,2018-01-07,1.0.0,4.0,0,True,False,False,True,False,False,False,False
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,0.0,Art & Design;Pretend Play,2018-01-15,2.0.0,4.0,0,True,False,False,True,False,False,False,False
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,0.0,Art & Design,2018-08-01,1.2.4,4.0,0,True,False,False,True,False,False,False,False
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,0.0,Art & Design,2018-06-08,Varies with device,4.2,0,True,False,False,False,False,False,True,False
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,0.0,Art & Design;Creativity,2018-06-20,1.1,4.4,0,True,False,False,True,False,False,False,False


In [54]:
df.drop(columns=['Genres', 'Current Ver'], axis=1, inplace=True)

In [55]:
df.head()

,App,Category,Rating,Reviews,Size,Installs,Price,Last Updated,Android Ver,Category_encoded,Type_Free,Type_Paid,Content Rating_Adults only 18+,Content Rating_Everyone,Content Rating_Everyone 10+,Content Rating_Mature 17+,Content Rating_Teen,Content Rating_Unrated
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,0.0,2018-01-07,4.0,0,True,False,False,True,False,False,False,False
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,0.0,2018-01-15,4.0,0,True,False,False,True,False,False,False,False
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,0.0,2018-08-01,4.0,0,True,False,False,True,False,False,False,False
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,0.0,2018-06-08,4.2,0,True,False,False,False,False,False,True,False
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,0.0,2018-06-20,4.4,0,True,False,False,True,False,False,False,False


### Year

In [57]:
df['Last Updated'] = pd.to_datetime(df['Last Updated'])
df['last_updated_year'] = df['Last Updated'].dt.year
df['last_updated_month'] = df['Last Updated'].dt.month

In [58]:
df.head()

,App,Category,Rating,Reviews,Size,Installs,Price,Last Updated,Android Ver,Category_encoded,Type_Free,Type_Paid,Content Rating_Adults only 18+,Content Rating_Everyone,Content Rating_Everyone 10+,Content Rating_Mature 17+,Content Rating_Teen,Content Rating_Unrated,last_updated_year,last_updated_month
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,0.0,2018-01-07,4.0,0,True,False,False,True,False,False,False,False,2018,1
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,0.0,2018-01-15,4.0,0,True,False,False,True,False,False,False,False,2018,1
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,0.0,2018-08-01,4.0,0,True,False,False,True,False,False,False,False,2018,8
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,0.0,2018-06-08,4.2,0,True,False,False,False,False,False,True,False,2018,6
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,0.0,2018-06-20,4.4,0,True,False,False,True,False,False,False,False,2018,6


# Save the csv

In [59]:
filepath = Path('data/googleplaystore_model_cleaned.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(filepath, index=False)

# Commits

In [63]:
commit_message = "initial cleaning"

subprocess.run(["powershell", "-File", "run_versioning_model_data.ps1", commit_message], check=True)

CompletedProcess(args=['powershell', '-File', 'run_versioning_model_data.ps1', 'initial cleaning'], returncode=0)

In [65]:
dvc_file = "data/googleplaystore_model_cleaned.csv.dvc"
gitignore_file = "data/.gitignore"
subprocess.run(["git", "add", dvc_file, gitignore_file, 'run_versioning_model_data.ps1', '.dvc/config'
                'version_model_data.py', 'README.md', 'clean_for_model.ipynb', '.gitignore'], check=True)

CalledProcessError: Command '['git', 'add', 'data/googleplaystore_model_cleaned.csv.dvc', 'data/.gitignore', 'run_versioning_model_data.ps1', '.dvc/configversion_model_data.py', 'README.md', 'clean_for_model.ipynb', '.gitignore']' returned non-zero exit status 128.